In [23]:
import pandas as pd

In [24]:
df = pd.read_csv('validations-nombre-2023-S1.csv', sep=';')

df['JOUR'] = pd.to_datetime(df['JOUR'], format='%Y-%m-%d')

In [25]:
vacances = [
    (pd.to_datetime("01/01/2023", format='%d/%m/%Y'), pd.to_datetime("03/01/2023", format='%d/%m/%Y')),
    (pd.to_datetime("18/02/2023", format='%d/%m/%Y'), pd.to_datetime("06/03/2023", format='%d/%m/%Y')),
    (pd.to_datetime("22/04/2023", format='%d/%m/%Y'), pd.to_datetime("09/05/2023", format='%d/%m/%Y')),
    (pd.to_datetime("08/07/2023", format='%d/%m/%Y'), pd.to_datetime("31/12/2023", format='%d/%m/%Y'))
]

jours_feries = [
    "01/01/2023",
    "10/04/2023",
    "01/05/2023",
    "08/05/2023",
    "18/05/2023",
    "19/05/2023", # Pont
    "29/05/2023"
]

def is_vacances(date):
    for debut, fin in vacances:
        if debut <= date <= fin:
            return True
    return False

def is_jours_feries(date):
    return date.strftime('%d/%m/%Y') in jours_feries

def is_johv(date):
    return date.weekday() < 5 and not is_vacances(date) and not is_jours_feries(date)

def is_sahv(date):
    return date.weekday() == 5 and not is_vacances(date) and not is_jours_feries(date)

def is_jovs(date):
    return date.weekday() < 5 and is_vacances(date)

def is_savs(date):
    return date.weekday() == 5 and is_vacances(date)

def is_dijfp(date):
    return date.weekday() == 6 or is_jours_feries(date)


In [26]:
df['CAT_JOUR'] = df['JOUR'].apply(lambda x: 'JOHV' if is_johv(x) else 'SAHV' if is_sahv(x) else 'JOVS' if is_jovs(x) else 'SAVS' if is_savs(x) else 'DIJFP')


In [27]:
# Compute the mean NB_VALID for each LIBELLE_ARRET and CAT_JOUR
df_mean = df.groupby(['LIBELLE_ARRET', 'CAT_JOUR'])['NB_VALD'].mean().reset_index()
df_mean = df_mean.pivot(index='LIBELLE_ARRET', columns='CAT_JOUR', values='NB_VALD')
df_mean.to_csv('moyenne-vald-arret-jour.csv', sep=';')

In [28]:
df_mean

CAT_JOUR,DIJFP,JOHV,JOVS,SAHV,SAVS
LIBELLE_ARRET,,,,,
ABBESSES,374.948148,449.404840,407.928889,555.128655,499.388889
ABLON,75.153846,208.990196,177.868687,100.108108,115.021739
ACHERES GRAND,8.987097,10.248288,9.664122,8.242105,8.318182
ACHERES VILLE,232.025000,633.365884,552.940000,358.561290,358.551020
AEROPORT CDG1,415.680162,829.011723,817.736842,445.229299,453.431818
...,...,...,...,...,...
VOSVES,5.142857,8.955263,8.403846,5.243243,5.666667
VULAINES SUR SE,9.228070,11.183794,10.437500,9.073529,8.241379
WAGRAM,253.985130,723.466960,616.497778,418.964912,360.111111
